# Gene expression & surface protein TIL 

### Load adata from ICI treated samples TIL 

In [1]:
# Libraries
import scanpy as sc
import anndata as ad
import pandas as pd
import sc_atlas_helpers as ah
import numpy as np
from scipy.stats import median_abs_deviation
from matplotlib.pyplot import rc_context
import matplotlib as plt

In [2]:
resDir ="/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/results/40_gex_surface_prot/"

In [3]:
adata_10mix_ICI1 = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/10mix-ICI1/outs/per_sample_outs/10mix-ICI1/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)
adata_10mix_ICI2 = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/10mix-ICI2/outs/per_sample_outs/10mix-ICI2/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)
adata_11mix_ICI1 = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/11mix-ICI1/outs/per_sample_outs/11mix-ICI1/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)
adata_11mix_ICI2 = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/11mix-ICI2/outs/per_sample_outs/11mix-ICI2/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)
adata_GF_ICI1 = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI1/outs/per_sample_outs/GF-ICI1/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)
adata_GF_ICI2 = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI2/outs/per_sample_outs/GF-ICI2/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)
adata_GF_ICI1_plus = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI1-plus/outs/per_sample_outs/GF-ICI1-plus/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)
adata_GF_ICI2_plus = sc.read_10x_h5("/data/projects/2021/MicrobialMetabolites/single-cell-sorted-cd8/2021-02-01_sorted_cd8_til/analyses_icbi/GF-ICI2-plus/outs/per_sample_outs/GF-ICI2-plus/count/sample_filtered_feature_bc_matrix.h5", gex_only=False)

/home/kvalem/.conda/envs/scanpy/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/kvalem/.conda/envs/scanpy/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/kvalem/.conda/envs/scanpy/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/kvalem/.conda/envs/scanpy/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/kvalem/.conda/envs/scanpy/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarn

### Add metadata

In [4]:
adata_10mix_ICI1.obs['adata_name'] = 'adata_10mix_ICI1'
adata_10mix_ICI2.obs['adata_name'] =  "adata_10mix_ICI2"
adata_11mix_ICI1.obs['adata_name'] = "adata_11mix_ICI1"
adata_11mix_ICI2.obs['adata_name'] = "adata_11mix_ICI2"
adata_GF_ICI1.obs['adata_name'] = "adata_GF_ICI1"
adata_GF_ICI2.obs['adata_name'] = "adata_GF_ICI2"
adata_GF_ICI1_plus.obs['adata_name'] = "adata_GF_ICI1_plus"
adata_GF_ICI2_plus.obs['adata_name'] =  "adata_GF_ICI2_plus"

In [5]:
def clean_adata(adata): 
    adata.var_names_make_unique()
    adata.layers["counts"] = adata.X.copy()
    #sc.pp.filter_genes(adata, min_counts=1)
    return adata

In [6]:
adata_10mix_ICI1 = clean_adata(adata_10mix_ICI1)
adata_10mix_ICI2 =  clean_adata(adata_10mix_ICI2)
adata_11mix_ICI1 =  clean_adata(adata_11mix_ICI1)
adata_11mix_ICI2 =  clean_adata(adata_11mix_ICI2)
adata_GF_ICI1 =  clean_adata(adata_GF_ICI1)
adata_GF_ICI2 =  clean_adata(adata_GF_ICI2)
adata_GF_ICI1_plus=  clean_adata(adata_GF_ICI1_plus)
adata_GF_ICI2_plus=  clean_adata(adata_GF_ICI2_plus)

In [7]:
def metadata(adata):
    print(list(set(adata.obs.adata_name)))
    adata.obs["condition"] = list(set(adata.obs.adata_name))[0].split("_")[1] #condition
    adata.obs["batch_id"] = list(set(adata.obs.adata_name))[0].split("_")[2] # batch_id
    adata.obs["sample_id"] = list(set(adata.obs.adata_name))[0].split("_",1)[1] # sample_id

In [8]:
adatas = []
adatas.append(adata_10mix_ICI1)
adatas.append(adata_10mix_ICI2)
adatas.append(adata_11mix_ICI1)
adatas.append(adata_11mix_ICI2)
adatas.append(adata_GF_ICI1)
adatas.append(adata_GF_ICI2)
adatas.append(adata_GF_ICI1_plus)
adatas.append(adata_GF_ICI2_plus)

In [9]:
for i in adatas:
    metadata(i)

['adata_10mix_ICI1']
['adata_10mix_ICI2']
['adata_11mix_ICI1']
['adata_11mix_ICI2']
['adata_GF_ICI1']
['adata_GF_ICI2']
['adata_GF_ICI1_plus']
['adata_GF_ICI2_plus']


In [10]:
adatas

[AnnData object with n_obs × n_vars = 11401 × 32292
     obs: 'adata_name', 'condition', 'batch_id', 'sample_id'
     var: 'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence'
     layers: 'counts',
 AnnData object with n_obs × n_vars = 10828 × 32292
     obs: 'adata_name', 'condition', 'batch_id', 'sample_id'
     var: 'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence'
     layers: 'counts',
 AnnData object with n_obs × n_vars = 9974 × 32292
     obs: 'adata_name', 'condition', 'batch_id', 'sample_id'
     var: 'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence'
     layers: 'counts',
 AnnData object with n_obs × n_vars = 9099 × 32292
     obs: 'adata_name', 'condition', 'batch_id', 'sample_id'
     var: 'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence'
     layers: 'counts',
 AnnData object with n_obs × n_vars = 8299 × 32292
     obs: 'adata_name', 'condition', 'batch_id', 'sample_id'
     var: 'gene_ids', 'feature

In [12]:
adata_merged = ad.concat(adatas,merge="same")

/home/kvalem/.conda/envs/scanpy/lib/python3.12/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [13]:
adata_merged.var[adata_merged.var["gene_ids"]=="ENSMUSG00000023078"]

,gene_ids,feature_types,genome,pattern,read,sequence
Cxcl13,ENSMUSG00000023078,Gene Expression,mm10,,,


In [16]:
adata_merged.var[adata_merged.var["gene_ids"]=="ENSMUSG00000060594"]

,gene_ids,feature_types,genome,pattern,read,sequence
Layn,ENSMUSG00000060594,Gene Expression,mm10,,,


In [14]:
adata_merged

AnnData object with n_obs × n_vars = 72592 × 32290
    obs: 'adata_name', 'condition', 'batch_id', 'sample_id'
    var: 'gene_ids', 'feature_types', 'genome', 'pattern', 'read', 'sequence'
    layers: 'counts'

In [15]:
adata = adata_merged

In [17]:
adata.write_h5ad(f"{resDir}adata_merged_til_all_genes.h5ad")